# Data & data wrangling

## Source and access
Data used in this project were downloaded from kaggle (https://www.kaggle.com/dhruvildave/new-york-times-best-sellers). Data can be freely downloaded into a csv file. The following script clean and convert the data into a text file. Ready to be used in the main script.

In [15]:
### Packages needed ###
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
### if need to install pacakges use, uncomment respective line
#%pip install pandas
#%pip install numpy
#%pip install datetime

## Original data

In [16]:
# load data into a pandas data frame
data = pd.read_csv('data/bestsellers.csv')
data.head(2)

,published_date,list_name,list_name_encoded,rank,isbn13,isbn10,title,author,description,amazon_product_url,price,weeks_on_list
0,2010-01-03,Chapter Books,chapter-books,1,9780316036245,0316036242,WITCH AND WIZARD,James Patterson and Gabrielle Charbonnet,"One of each, brother and sister, flex their ne...",https://www.amazon.com/Witch-Wizard-James-Patt...,17.99,1
1,2010-01-03,Chapter Books,chapter-books,2,9780439023481,0439023483,THE HUNGER GAMES,Suzanne Collins,"In a dystopian future, a girl fights for survi...",https://www.amazon.com/The-Hunger-Games-Suzann...,17.99,67


## cleaning of original data

In [17]:
# list of columns name to be dropped
colnames_to_drop = ['list_name_encoded', 'list_name','rank','isbn13', 'isbn10', 'amazon_product_url']

#drop coloumn names
data.drop(columns = colnames_to_drop, axis = 1,inplace = True)
data.head()

,published_date,title,author,description,price,weeks_on_list
0,2010-01-03,WITCH AND WIZARD,James Patterson and Gabrielle Charbonnet,"One of each, brother and sister, flex their ne...",17.99,1
1,2010-01-03,THE HUNGER GAMES,Suzanne Collins,"In a dystopian future, a girl fights for survi...",17.99,67
2,2010-01-03,CATCHING FIRE,Suzanne Collins,"The protagonist of ""The Hunger Games"" returns.",17.99,16
3,2010-01-03,THE MAGICIAN’S ELEPHANT,Kate DiCamillo,An orphan in search of his sister follows a fo...,16.99,15
4,2010-01-03,FALLEN,Lauren Kate,"Thwarted love among misfits at a Savannah, Ga....",17.99,2


### Issue with data
In the original dataset, some lines contains comma (e.g. in description or title), therefore we cannot just save the file into a txt. format separated by comma. This would result in a txt file that as varying numbers of columns and would be unusable for later stages. We decide to separate the columns using ";". But before doing this, we check all columns if there are ";" in the strings. If yes, we replace them by different signs depending on the columns. Here is the rule:

- All ";" in title column were just removed.
- All ";" in author column were replace by the "|" sign.
- All ";" in description column were replace by the "|" sign.

Besides this, we also remove all "\n" existing in the description column.



In [18]:
## find & replace element of dataframe in order that dataframe can be correclty save into a csv or txt file
# using the apply() and replace(), this can be done easily
data['title'] = data['title'].apply(lambda x: str(x).replace(';',''))
data['author'] = data['author'].apply(lambda x: str(x).replace(';','|'))
data['description'] = data['description'].apply(lambda x: str(x).replace(';',' |'))
data['description'] = data['description'].apply(lambda x: str(x).replace('\n',''))

# print data type of all columns for information
print(data.dtypes)
data.head()

published_date     object
title              object
author             object
description        object
price             float64
weeks_on_list       int64
dtype: object


,published_date,title,author,description,price,weeks_on_list
0,2010-01-03,WITCH AND WIZARD,James Patterson and Gabrielle Charbonnet,"One of each, brother and sister, flex their ne...",17.99,1
1,2010-01-03,THE HUNGER GAMES,Suzanne Collins,"In a dystopian future, a girl fights for survi...",17.99,67
2,2010-01-03,CATCHING FIRE,Suzanne Collins,"The protagonist of ""The Hunger Games"" returns.",17.99,16
3,2010-01-03,THE MAGICIAN’S ELEPHANT,Kate DiCamillo,An orphan in search of his sister follows a fo...,16.99,15
4,2010-01-03,FALLEN,Lauren Kate,"Thwarted love among misfits at a Savannah, Ga....",17.99,2


In [19]:
## If data wanted to be save data in csv or parquet format before last step, uncomment lines
#data.to_csv("data/NYTB_full.csv", sep = ";", index = False, header = True)
#data.to_parquet('data/NYTB_full.par')

The following part removes all duplicates in the data frame and sort the rows by alphabetic order.
Duplicates stem from the fact that books which are many weeks on the NYT bestseller list will be on listed on the data frame with an new number of weeks on the NYT bestseller list and with a new release date (published date). We remove all duplicates. We keep the first release date and the maximal amount of week on the bestseller list. All other information are repeated so it does not matter which one is kept. To do so, we split the data frame into 3 sub data frames and sort them by title and relevant variable, then remove duplicates. After doing this three times, we merge the three sub data frames together to reconstruct one single data frame with all information needed.

In [20]:
# 1st dataframe with only 4 columns: title, published date (PD), author (A) and descriptition (D)
PDAD = data.loc[:,['title','published_date','author','description']]

# sort values of by title and and then by published date + set title as index of data frame
PDAD = PDAD.sort_values(['title','published_date'], ascending = [True, True]).set_index('title')
print("number of observation before removing duplicates:", len(PDAD))

# remove all duplicated in index (here title) and keep only the first one (with oldest published date)
PDAD = PDAD[PDAD.index.duplicated(keep="first")==False].reset_index()
print("number of observation after removing duplicates:", len(PDAD))

PDAD.head(5)

number of observation before removing duplicates: 61430
number of observation after removing duplicates: 6577


,title,published_date,author,description
0,#ASKGARYVEE,2016-04-03,Gary Vaynerchuk,nan
1,#GIRLBOSS,2014-05-25,Sophia Amoruso,nan
2,10% HAPPIER,2014-03-30,Dan Harris,"A co-anchor of ""Nightline"" reports on the scie..."
3,10-DAY GREEN SMOOTHIE CLEANSE,2014-08-03,J J Smith,nan
4,"100 BULLETS: DELUXE EDITION, BOOK FOUR",2013-05-05,Brian Azzarello and Eduardo Risso,"This deluxe edition of the noir series, about ..."


In [21]:
# 2nd data frame with only title and price 
price = data[['title', 'price']]

# sort values of by title and and then by price (descending) + set title as index of data frame
price = price.sort_values(['title','price'], ascending = [True, False]).set_index('title')
print("number of observation before removing duplicates:", len(price))

# remove all duplicated in index (here title) and keep only the first one (with highest price)
price = price[price.index.duplicated(keep="first")==False].replace(0,'nan').reset_index()
print("number of observation after removing duplicates:", len(price))
price.head(5)

number of observation before removing duplicates: 61430
number of observation after removing duplicates: 6577


,title,price
0,#ASKGARYVEE,nan
1,#GIRLBOSS,nan
2,10% HAPPIER,nan
3,10-DAY GREEN SMOOTHIE CLEANSE,nan
4,"100 BULLETS: DELUXE EDITION, BOOK FOUR",49.99


In [22]:
# 3rd data frame with numbers of weeks on bestseller list and title
week_on_best = data[['title', 'weeks_on_list']]

# sort values of by title and and then by number of weeks (descending) + set title as index of data frame
week_on_best = week_on_best.sort_values(['title','weeks_on_list'], ascending = [True, False]).set_index('title')
print("number of observation before removing duplicates:", len(week_on_best))

# remove all duplicated in index (here title) and keep only the first one (with highest number of weeks)
week_on_best = week_on_best[week_on_best.index.duplicated(keep="first")==False].reset_index()
print("number of observation after removing duplicates:", len(week_on_best))
week_on_best.head(5)

number of observation before removing duplicates: 61430
number of observation after removing duplicates: 6577


,title,weeks_on_list
0,#ASKGARYVEE,0
1,#GIRLBOSS,14
2,10% HAPPIER,14
3,10-DAY GREEN SMOOTHIE CLEANSE,40
4,"100 BULLETS: DELUXE EDITION, BOOK FOUR",4


In [26]:
## merging all three sub data frame into one single dataframe with without duplicated titles
# we merge the 1st and 2nd data frame on the column title
df = PDAD.merge(price, left_on = 'title', right_on = 'title')

# We join the 3rd data frame to the previously merge one, again on title
df = df.join(week_on_best.set_index('title'), on = 'title')
print('number of observaton after merger',len(df))
print('(should be exactly the same as in sub data frames)')
df.head()

number of observaton after merger 6577
(should be exactly the same as in sub data frames)


,title,published_date,author,description,price,weeks_on_list
0,#ASKGARYVEE,2016-04-03,Gary Vaynerchuk,nan,nan,0
1,#GIRLBOSS,2014-05-25,Sophia Amoruso,nan,nan,14
2,10% HAPPIER,2014-03-30,Dan Harris,"A co-anchor of ""Nightline"" reports on the scie...",nan,14
3,10-DAY GREEN SMOOTHIE CLEANSE,2014-08-03,J J Smith,nan,nan,40
4,"100 BULLETS: DELUXE EDITION, BOOK FOUR",2013-05-05,Brian Azzarello and Eduardo Risso,"This deluxe edition of the noir series, about ...",49.99,4


In [9]:
# save the cleaned data frame into a new csv file called 'NYTB_clean'
df.to_csv("data/NYTB_clean.csv", sep = ";", index = False, header = True)